## Notebook to perform text detection + recognition + geocoder queries

1. Install required libraries
2. Download pretrained models
3. (optional) preprocess and extract map area
4. Split into tiles
5. Text detection + recognition
6. Postprocess detections
7. Geocoder querying

In [2]:
import os
import json
import re
import random
import requests
import urllib.parse
import time

import numpy as np
from matplotlib import pyplot as plt
import cv2
from scipy import spatial
import tensorflow as tf
import keras_ocr
from shapely import geometry
from fuzzywuzzy import fuzz

### Download and initialize text detection & recognition models

These will be saved in the folder: pretrained_models

In [3]:
# Initialize text detector + recognizer
pipeline = keras_ocr.pipeline.Pipeline()

Looking for pretrained_models/craft_mlt_25k.h5
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Looking for pretrained_models/crnn_kurapan.h5


In [4]:
def extract_map_area(img, verbose=0):
    """
    Extract the coordinates of the black border surrounding the actual map in the image
    :param img:
    :param verbose:
    :return:
    """
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)

    ret, thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # Create structure element for extracting hor/ver lines through morphology operations
    size = 100
    horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (size, 1))
    verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, size))
    # Apply morphology operations
    horizontal = cv2.erode(thresh, horizontalStructure)
    horizontal = cv2.dilate(horizontal, horizontalStructure)

    vertical = cv2.erode(thresh, verticalStructure)
    vertical = cv2.dilate(vertical, verticalStructure)
    
    merged = horizontal | vertical
    merged = cv2.dilate(merged,np.ones((3,3)))
    
    rows, cols = merged.shape

    top_lines = []
    bot_lines = []
    left_lines = []
    right_lines = []

    max_pixel_dist = 2
    # Minimum padding for box location
    pad_right = 100
    pad_left = 25
    pad_top = 100
    pad_bot = 200

    for r in range(pad_top, rows - pad_bot):
        nonzero = cv2.countNonZero(merged[r, :])
        # If atleast 1/8 is a line
        if nonzero > cols // 8:

            # Top
            if r < rows // 2:
                current_lines = top_lines
            else:
                current_lines = bot_lines

            if len(current_lines) == 0:
                # start_stop and count
                current_lines.append([r, r, nonzero])
            # If in range(forms a line)
            # Add the right coordinate and make sum of total pixels
            elif current_lines[-1][1] + max_pixel_dist >= r:
                current_lines[-1][1] = r
                current_lines[-1][2] += nonzero
                # Found a new line outside range of previous
            # Start a new one
            else:
                current_lines.append([r, r, nonzero])
                # print('Found a line outside of pixel dist', 'r', r)

    for c in range(pad_left, cols - pad_right):
        nonzero = cv2.countNonZero(merged[:, c])
        # If atleast half is white
        if nonzero > rows // 8:

            # Left
            if c < cols // 2:
                current_lines = left_lines
            else:
                current_lines = right_lines

            if len(current_lines) == 0:
                # start_stop and count
                current_lines.append([c, c, nonzero])
            # If in range(forms a line)
            # Add the right coordinate and make sum of total pixels
            elif current_lines[-1][1] + max_pixel_dist >= c:
                current_lines[-1][1] = c
                current_lines[-1][2] += nonzero
                # Found a new line outside range of previous
            # Start a new one
            else:
                current_lines.append([c, c, nonzero])
                # print('Found a line outside of pixel dist', 'c', c)

    # Take the best lines if there are multiple
    left_lines = sorted(left_lines, key=lambda x: (x[1] - x[0], x[2]), reverse=True)
    right_lines = sorted(right_lines, key=lambda x: (x[1] - x[0], x[2]), reverse=True)
    top_lines = sorted(top_lines, key=lambda x: (x[1] - x[0], x[2]), reverse=True)
    bot_lines = sorted(bot_lines, key=lambda x: (x[1] - x[0], x[2]), reverse=True)
    
    
    x1, y1 = left_lines[0][0], top_lines[0][0]
    x2, y2 = right_lines[0][1], bot_lines[0][1]

    if verbose:
        print('x1,y1', x1, y1, '\tx2,y2', x2, y2)
        out = img.copy()
        cv2.rectangle(out, (x1, y1), (x2, y2), (255, 0, 0), 40)
        plt.figure(figsize=(12,12))
        plt.imshow(out)
        plt.show()
        
    return x1,y1,x2,y2
    

def convert_predictions(predictions):
    """
    Convert predictions to object style, so it works with previously developed code
    """
    res = []
    for label,bbox in predictions:
        bbox = np.array(bbox,dtype=np.int32).flatten().tolist()
        res.append({'text':label, 'boundingBox':bbox})
    return res


def map_coords(predictions,x1,y1):
    """
    Translate coordinates with x1,y1
    """
    if x1+y1 == 0:
        return predictions
    
    res = []
    for label,bbox in predictions:
        bbox = np.array(bbox,dtype=np.int32)
        bbox[:,0] += x1
        bbox[:,1] += y1
        res.append((label,bbox))
        
    return res

def map_coords_word(word,x1,y1):
    """
    Translate coordinates with x1,y1
    """
    if x1 + y1 == 0:
        return word

    bbox = word['boundingBox']
    bbox = [bbox[i] + x1 if i % 2 == 0 else bbox[i] + y1 for i in range(len(bbox))]
    word['boundingBox'] = bbox

    return word
    


def get_center(bbox):
    """
    Returns the center of the given bbox
    :param bbox:
    :return:
    """
    center_x = np.mean(bbox[::2])
    center_y = np.mean(bbox[1::2])
    return [center_x,center_y]
    

def merge_coords(preds):
    """
    Merge the coordinates into one predictions (lines containing e or n)
    """
    res = preds.copy()
    ids_to_remove = set()
    x_dist = 150
    for i in range(len(preds)):
        p = preds[i]
        text = p['text']
        
        # Check for numbers to the left of this box
        # These should always be the preceding boxes
        if text == 'e' or text == 'n':
            bbox = p['boundingBox']
            center = get_center(bbox)
            min_y = np.min(bbox[1::2])
            max_y = np.max(bbox[1::2])
            min_x = center[0] - x_dist
            max_x = center[0]
            matches = []
            
            for j in range(i-3,i):
                p2 = preds[j]
                bbox2 = p2['boundingBox']
                center2 = get_center(bbox2)
                if min_x < center2[0] < max_x and min_y < center2[1] < max_y:
                    matches.append(p2)
                    ids_to_remove.add(j)
            
            if len(matches) > 0:
                matches.append(p)
                merged = merge_matches(matches)
                res[i] = merged
                
                
    for id_ in sorted(ids_to_remove,reverse=True):
        del res[id_]
                    
    return res

def merge_bbox(b1, b2):
    """
    Merge the two given bboxes
    :param b1:
    :param b2:
    :return:
    """
    # X top left, Y top left, X top right, Y top right, X bottom right, Y bottom right, X bottom left, Y bottom left
    return [min(b1[0], b2[0]), min(b1[1], b2[1]), max(b1[2], b2[2]), min(b1[3], b2[3]),
            max(b1[4], b2[4]), max(b1[5], b2[5]), min(b1[6], b2[6]), max(b1[7], b2[7])]

def merge_matches(matches):
    """
    Merge the predictions from start to end
    """
    merged_text = ''
    merged_bbox = None
    for m in matches:
        bbox = m['boundingBox']
        text = m['text']
        
        if merged_bbox is None:
            merged_bbox = bbox
            merged_text = text
        else:
            merged_bbox = merge_bbox(merged_bbox,bbox)
            merged_text += text
            
    return {'text':merged_text,'boundingBox':merged_bbox}

def filter_lines(preds):
    """
    Split predictions into those around the map area (greenwich and coordinates)
    and those signifying the x/y coordinates on top. Remove the rest
    :param lines:
    :return:
    """
    res = []
    res_degrees = []
    for pred in preds:

        text = pred['text']

        # If we find greenwich in text, we add it to degrees
        if 'greenwich' in text:
            pred['center'] = get_center(pred['boundingBox'])
            res_degrees.append(pred)
            continue
        
        # If we find n or e in text, we check if the rest is a number, if it is, we add it
        # to degrees
        if 'n' in text:
            without = text.replace('n','')
            try:
                number = int(without)
                res_degrees.append(pred)
                continue
            except:
                pass
        if 'e' in text:
            without = text.replace('e','')
            try:
                number = int(without)
                res_degrees.append(pred)
                continue
            except:
                pass
        
        # Now we check the rest, if it is a string of length 3 and contains at least 2 numbers
        if len(text) == 3 and sum(char.isdigit() for char in text) > 1:
            pred['center'] = get_center(pred['boundingBox'])
            res.append(pred)
    
    return res, res_degrees

def rotate_predictions(preds,crop_length,crop_width,side='left'):
    """
    Rotate predictions back into coordinates of original crop
    """
    res = []
    
    for label,bbox in preds:
        if side== 'left':
            bbox = np.array(bbox,np.int32)
            bbox = np.fliplr(bbox)
            bbox[:,1] = crop_length - bbox[:,1]
            
        elif side == 'right':
            bbox = np.array(bbox,np.int32)
            bbox = np.fliplr(bbox)
            bbox[:,0] = crop_width - bbox[:,0]
        
        res.append((label,bbox))
    
    return res


def line_correct(lines_top, lines_bot, lines_left, lines_right):
    """
    Correct the errors in the coordinates in the detected lines
    :param lines_top:
    :param lines_bot:
    :param lines_left:
    :param lines_right:
    :return:
    """
    # sort lines in asc order
    top = sorted(lines_top, key=lambda x: x['center'][0])
    bot = sorted(lines_bot, key=lambda x: x['center'][0])
    left = sorted(lines_left, key=lambda x: x['center'][1], reverse=True)
    right = sorted(lines_right, key=lambda x: x['center'][1], reverse=True)
    
    # Replace o by 0
    for l in (top,bot,left,right):
        for l2 in l:
            l2['text'] = l2['text'].replace('o','0')
    
    print('top lines:',[t['text'] for t in top])
    print('bot lines:',[t['text'] for t in bot])
    print('left lines:',[t['text'] for t in left])
    print('right lines:',[t['text'] for t in right])
    # Correct top & bot
    avg_factor_top, avg_p_t, avg_c_t = get_coord_dist(top, 0)
    avg_factor_bot, avg_p_b, avg_c_b = get_coord_dist(bot, 0)

    x_factor = np.mean((avg_factor_top, avg_factor_bot))
    avg_p_x = np.mean((avg_p_t, avg_p_b))
    avg_c_x = np.mean((avg_c_t, avg_c_b))

    # Correct left & right
    avg_factor_left, avg_p_l, avg_c_l = get_coord_dist(left, 1)
    avg_factor_right, avg_p_r, avg_c_r = get_coord_dist(right, 1)
    y_factor = np.mean((avg_factor_left, avg_factor_right))

    avg_p_y = np.mean((avg_p_l, avg_p_r))
    avg_c_y = np.mean((avg_c_l, avg_c_r))

    conversion_factor = np.array((-y_factor, x_factor))
    avg_point = np.array((avg_p_y, avg_p_x))
    avg_coords = np.array((avg_c_y, avg_c_x))
    zero_point_coords = avg_coords - (avg_point * conversion_factor)

    return zero_point_coords, conversion_factor


def get_coord_dist(lines, center_index):
    """
    Get the average coordinate distance over pixel distance factor, average pixel coordinate, and average coordinate
    :param lines:
    :param center_index:
    :return:
    """
    coords = []
    pixel_coords = []
    for l in lines:
        c = l['text']
        try:
            c = int(c)
            coords.append(c)
            pixel_coords.append(l['center'][center_index])
        except ValueError:
            pass

    median_c = np.median(coords)
    ids_to_remove = []
    # remove outliers
    for i, c in enumerate(coords):
        if abs(c - median_c) > 10:
            ids_to_remove.append(i)
    for i in sorted(ids_to_remove, reverse=True):
        del coords[i]
        del pixel_coords[i]

    d_coords = np.abs(np.diff(coords))
    d_pixel_coords = np.abs(np.diff(pixel_coords))

    return np.mean(d_coords / d_pixel_coords), np.mean(pixel_coords), np.mean(coords)


def extract_coordinate_area(img,map_info,x1,y1,x2,y2):
    """
    Extract the actual map area and the pixel to coordinate transform
    """
    crop_width = 150
    crop_length = 2000
    # crop of whole top

    all_preds = []

    # Top and bottom first
    current_x = x1
    while current_x < x2:
        print('Current x:',current_x)
        x_stop = min(x2, current_x + crop_length)

        crop_top = img[y1:y1 + crop_width, current_x:x_stop]
        preds = pipeline.recognize([crop_top])[0]
        preds = map_coords(preds, current_x, y1)
        preds = convert_predictions(preds)
        all_preds.extend(preds)

        crop_bot = img[y2 - crop_width:y2, current_x:x_stop]
        preds = pipeline.recognize([crop_bot])[0]
        preds = map_coords(preds, current_x, y2 - crop_width)
        preds = convert_predictions(preds)
        all_preds.extend(preds)
        current_x += crop_length

    # Filter out lines with numbers
    all_lines = merge_coords(all_preds)
    filtered_lines, degrees_lines = filter_lines(all_lines)
    
    
    top_lines = [l for l in filtered_lines if l['center'][1] < (y2 - y1) // 2]
    bot_lines = [l for l in filtered_lines if l['center'][1] > (y2 - y1) // 2]

    delta_y = 10

    possible_lines_top = []
    possible_ids_top = []
    possible_lines_bot = []
    possible_ids_bot = []

    top_y = [l['center'][1] for l in top_lines]
    bot_y = [l['center'][1] for l in bot_lines]

    # TODO Fix code duplication

    for i, y in enumerate(top_y):
        if len(possible_lines_top) == 0:
            possible_lines_top = [[y]]
            possible_ids_top = [[i]]
        else:

            average_y = [abs(y - np.mean(item)) for item in possible_lines_top]
            if min(average_y) < delta_y:
                idx = np.argmin(average_y)
                possible_lines_top[idx].append(y)
                possible_ids_top[idx].append(i)
            else:
                possible_lines_top.append([y])
                possible_ids_top.append([i])

    for i, y in enumerate(bot_y):
        if len(possible_lines_bot) == 0:
            possible_lines_bot = [[y]]
            possible_ids_bot = [[i]]
        else:

            average_y = [abs(y - np.mean(item)) for item in possible_lines_bot]
            if min(average_y) < delta_y:
                idx = np.argmin(average_y)
                possible_lines_bot[idx].append(y)
                possible_ids_bot[idx].append(i)
            else:
                possible_lines_bot.append([y])
                possible_ids_bot.append([i])

    ids_top = possible_ids_top[np.argmax([len(i) for i in possible_ids_top])]
    lines_top = [top_lines[i] for i in ids_top]
    ids_bot = possible_ids_bot[np.argmax([len(i) for i in possible_ids_bot])]
    lines_bot = [bot_lines[i] for i in ids_bot]

    # Do the same for left/right

    all_lines2 = []
    current_y = y1

    while current_y < y2:
        print('Current y', current_y)
        y_stop = min(y2, current_y + crop_length)

        crop_left = img[current_y:y_stop, x1:x1 + crop_width]
        # Rotate image so text is upright
        rot = np.rot90(crop_left,axes=(1,0))
        preds = pipeline.recognize([rot])[0]
        # Rotate predictions back to original image

        preds = rotate_predictions(preds,crop_length,crop_width,side='left')
        preds = map_coords(preds, x1, current_y)
        preds = convert_predictions(preds)
        all_lines2.extend(preds)

        crop_right = img[current_y:y_stop, x2 - crop_width:x2]
        # Rotate image so text is upright
        rot = np.rot90(crop_right)
        preds = pipeline.recognize([rot])[0]
        # Rotate predictions back to original image
        preds = rotate_predictions(preds,crop_length,crop_width,side='right')
        preds = map_coords(preds, x2 - crop_width,current_y)
        preds = convert_predictions(preds)
        all_lines2.extend(preds)

        current_y += crop_length
    
    
    all_lines2 = merge_coords(all_lines2)
    '''
    print('All lines (left/right):')
    for l in all_lines2:
        print(l['text'],get_center(l['boundingBox']))
    '''
    filtered_lines, degrees_lines2 = filter_lines(all_lines2)
    
    left_lines = [l for l in filtered_lines if l['center'][0] < (x2 - x1) // 2]
    right_lines = [l for l in filtered_lines if l['center'][0] > (x2 - x1) // 2]
    '''
    print('Filtered lines (left/right):')
    for l in filtered_lines:
        print(l['text'],l['center'])
    '''
    
    delta_x = 10

    possible_lines_left = []
    possible_ids_left = []
    possible_lines_right = []
    possible_ids_right = []

    left_x = [l['center'][0] for l in left_lines]
    right_x = [l['center'][0] for l in right_lines]

    for i, x in enumerate(left_x):
        if len(possible_lines_left) == 0:
            possible_lines_left = [[x]]
            possible_ids_left = [[i]]
        else:

            average_x = [abs(x - np.mean(item)) for item in possible_lines_left]
            if min(average_x) < delta_x:
                idx = np.argmin(average_x)
                possible_lines_left[idx].append(x)
                possible_ids_left[idx].append(i)
            else:
                possible_lines_left.append([x])
                possible_ids_left.append([i])

    for i, x in enumerate(right_x):
        if len(possible_lines_right) == 0:
            possible_lines_right = [[x]]
            possible_ids_right = [[i]]
        else:

            average_x = [abs(x - np.mean(item)) for item in possible_lines_right]
            if min(average_x) < delta_x:
                idx = np.argmin(average_x)
                possible_lines_right[idx].append(x)
                possible_ids_right[idx].append(i)
            else:
                possible_lines_right.append([x])
                possible_ids_right.append([i])

    ids_left = possible_ids_left[np.argmax([len(i) for i in possible_ids_left])]
    lines_left = [left_lines[i] for i in ids_left]
    ids_right = possible_ids_right[np.argmax([len(i) for i in possible_ids_right])]
    lines_right = [right_lines[i] for i in ids_right]

    # Determine pixel to Lambert72 conversion factors
    c_zero, factor = line_correct(lines_top, lines_bot, lines_left, lines_right)
    print('Coordinates for 0,0:', c_zero)
    map_info['c_zero'] = c_zero.tolist()
    map_info['factor'] = factor.tolist()


    # Now find the 'actual' map area by looking at the lines with degrees as they are closest to the actual map area

    for l in degrees_lines:
        l['center'] = get_center(l['boundingBox'])

    d_top = [l for l in degrees_lines if l['center'][1] < (y2 - y1) // 2]
    d_bot = [l for l in degrees_lines if l['center'][1] > (y2 - y1) // 2]
    
    if len(d_top) == 0:
        d_top = [l for l in get_backup_degrees_lines(all_lines,direction='up') if l['center'][1] < (y2 - y1) // 2]
    
    if len(d_bot) == 0:
        d_bot = [l for l in get_backup_degrees_lines(all_lines,direction='up') if l['center'][1] > (y2 - y1) // 2]
    
    # Here we try to detect greenwich or the degrees with N and E
    # to determine the effective map area

    # Find one correct degree
    found = False
    y_top = 0
    # Check for greenwich
    for l in d_top:
        if 'greenwich' in l['text'].lower():
            found = True
            y_top = np.max(l['boundingBox'][1::2])
            break

    # Check for N in combination with 6 digits for lines top
    if not found:
        for l in d_top:
            text = l['text'].lower()

            if text.endswith('n') and len(text) in {6,7}:
                # check if numbers are before it
                try:
                    n = int(text[:-1])
                    found = True
                    y_top = np.max(l['boundingBox'][1::2])
                    break
                except ValueError:
                    pass

    # Else take the average of min of the boxes
    if not found:
        for l in d_top:
            y_top += np.max(l['boundingBox'][1::2])
        y_top /= len(d_top)

    # Find one correct degree
    found = False
    y_bot = 0
    # Check for greenwich
    for l in d_bot:
        if 'greenwich' in l['text'].lower():
            found = True
            y_bot = np.min(l['boundingBox'][1::2])
            break
    # Check for ' or " in combination with -N for lines top
    if not found:
        for l in d_bot:
            text = l['text'].lower()
            if text.endswith('n') and len(text) in {6,7}:
                # check if numbers are before it
                try:
                    n = int(text[:-1])
                    found = True
                    y_bot = np.min(l['boundingBox'][1::2])
                    break
                except ValueError:
                    pass
    # Else take the average of min of the boxes
    if not found:
        for l in d_bot:
            y_bot += np.min(l['boundingBox'][1::2])
        y_bot /= len(d_bot)

    # Now left/right

    for l in degrees_lines2:
        l['center'] = get_center(l['boundingBox'])
    
    
    d_left = [l for l in degrees_lines2 if l['center'][0] < (x2 - x1) // 2]
    d_right = [l for l in degrees_lines2 if l['center'][0] > (x2 - x1) // 2]
    
    if len(d_left) == 0:
        d_left = [l for l in get_backup_degrees_lines(all_lines2,direction='left') if l['center'][0] < (x2 - x1) // 2]
    
    if len(d_right) == 0:
        d_right = [l for l in get_backup_degrees_lines(all_lines2,direction='left') if l['center'][0] > (x2 - x1) // 2]
    
    # Find one correct degree
    found = False
    x_left = 0

    # Check for ' or " in combination with -N or east for lines left
    if not found:
        for l in d_left:
            text = l['text'].lower()

            if text.endswith('e') and len(text) in {6,7}:
                # check if numbers are before it
                try:
                    n = int(text[:-1])
                    found = True
                    x_left = np.max(l['boundingBox'][::2])
                    break
                except ValueError:
                    pass

    # Else take the average of min of the boxes
    if not found:
        for l in d_left:
            x_left += np.max(l['boundingBox'][::2])
        x_left /= len(d_left)

    # Find one correct degree
    found = False
    x_right = 0

    # Check for ' or " in combination with -N for lines top
    if not found:
        for l in d_right:
            text = l['text'].lower()

            if text.endswith('e') and len(text) in {6,7}:
                # check if numbers are before it
                try:
                    n = int(text[:-1])
                    found = True
                    x_right = np.min(l['boundingBox'][::2])
                    break
                except ValueError:
                    pass

    # Else take the average of min of the boxes
    if not found:
        for l in d_right:
            x_right += np.min(l['boundingBox'][::2])
        x_right /= len(d_right)
        
    map_info['map_area'] = {'x1': int(x_left), 'y1': int(y_top), 'x2': int(x_right), 'y2': int(y_bot)}
    map_info['coord_lines_top'] = lines_top
    map_info['coord_lines_bot'] = lines_bot
    map_info['coord_lines_left'] = lines_left
    map_info['coord_lines_right'] = lines_right
    
    return map_info


def write_predictions(out_paths,prediction_groups):
    # if only 1 path and prediction is given
    if isinstance(out_paths,str):
        out_paths = [out_paths]
        prediction_groups = [prediction_groups]
            
    for out_path,pred_group in zip(out_paths,prediction_groups):
        words = []
        
        for text,bbox in pred_group:
            bbox = np.int32(np.round(bbox)).ravel().tolist()
            obj = {'text':text,'boundingBox':bbox}
            words.append(obj)
        
        json.dump({'words':words},open(out_path,'w',encoding='utf-8'))
        
def iou(a, b):
    a = geometry.Polygon([(a[0], a[1]), (a[2], a[3]), (a[4], a[5]), (a[6], a[7])])
    b = geometry.Polygon([(b[0], b[1]), (b[2], b[3]), (b[4], b[5]), (b[6], b[7])])

    return a.intersection(b).area / a.union(b).area


def get_backup_degrees_lines(lines,direction='up'):
    """
    Loosen restrictions to get degrees lines
    Just look for number combinations of length 4 or higher and look for n or e
    """
    degrees_lines = []
    char = 'n' if direction=='up' else 'e'
    for l in lines:
        text = l['text'].lower()
        center = get_center(l['boundingBox'])
        l['center'] = center
        
        if direction == 'up' and text == char:
            degrees_lines.append(l)
        
        elif len(text) > 3:
            try:
                num = int(text)
                degrees_lines.append(l)
            except:
                pass
    return degrees_lines
    

### Detect effective map region (not needed for TOP50Raster)

### Split into tiles (2500x2500 with 500 overlap)

In [5]:
data_dir = 'dataset/TOP50raster'
# if img_dir == crop_dir, no preprocessing will occurr
img_dir = os.path.join(data_dir,'cropped')
crop_dir = os.path.join(data_dir,'cropped')
json_dir = os.path.join(data_dir,'json')
tile_dir = os.path.join(data_dir,'tiles')

tile_size = 2000
overlap = 500

In [6]:
for fname in os.listdir(img_dir):
    if fname.endswith('.jpg'):
        
        map_name = fname.replace('.jpg','')
        print('Current map:',map_name)
        lines_fname = os.path.join(json_dir,fname.replace('.jpg','_lines.json'))
        
        # Check if the map has been fully processed before
        if os.path.exists(lines_fname):
            print('Map already processed')
            continue
        
        json_fname = os.path.join(json_dir,fname.replace('.jpg','.json'))
        try:
            map_info = json.load(open(json_fname,'r'))
        except FileNotFoundError:
            map_info = {}
        
        crop_path = os.path.join(img_dir,fname)
        orig_img = cv2.imread(os.path.join(img_dir,fname))
        
        # If map still needs to be cropped (not the case for TOP50raster)
        if not os.path.exists(crop_path):
            img = cv2.cvtColor(orig_img,cv2.COLOR_BGR2RGB)
            x1,y1,x2,y2 = extract_map_area(img,verbose=0)
            map_info = extract_coordinate_area(img,map_info,x1,y1,x2,y2)
            
            map_area = map_info['map_area']
            x1 = map_area['x1']
            y1 = map_area['y1']
            x2 = map_area['x2']
            y2 = map_area['y2']
            crop = orig_img[y1:y2,x1:x2]
            cv2.imwrite(crop_path,crop)
            json.dump(map_info, open(json_fname, 'w'))
        
        # Split into tiles and save them in tiles dir
        img = orig_img
        x1,x2,y1,y2 = 0,0,0,0
        h, w, _ = img.shape
        if y2 == 0:
            y2 = h
        if x2 == 0:
            x2 = w
        row_index = 0
        for r in range(y1, y2, tile_size):
            col_index = 0
            for c in range(x1, x2, tile_size):
                if r > h or c > w:
                    # This should never execute
                    print('Something went wrong')
                    continue

                r_max = min(y2, r + tile_size + overlap)
                c_max = min(x2, c + tile_size + overlap)
                tile = img[r:r_max, c:c_max]
                fname_tile = os.path.join(tile_dir, map_name + '_tile_' + str(row_index) + '_' + str(col_index) + '.jpg')
                cv2.imwrite(fname_tile, tile)
                col_index += 1
            row_index += 1
        print()
        

Current map: TOP50raster-31O-2018

Current map: TOP50raster-32O-2018

Current map: TOP50raster-32W-2018

Current map: TOP50raster-38O-2018

Current map: TOP50raster-39O-2018

Current map: TOP50raster-39W-2018

Current map: TOP50raster-44O-2018

Current map: TOP50raster-45O-2018

Current map: TOP50raster-45W-2018



### Detect and recognize text in each tile

In [7]:
for fname in os.listdir(img_dir):
    if fname.endswith('.jpg'):
        map_name = fname.replace('.jpg','')
        print('Current:',map_name)
        lines_fname = os.path.join(json_dir,fname.replace('.jpg','_lines.json'))
        
        # Check if the map has been fully processed before
        if os.path.exists(lines_fname):
            print('Map already processed')
            continue
        tile_fnames = [os.path.join(tile_dir,f) for f in os.listdir(tile_dir) if f.endswith('.jpg') and map_name in f]
        json_fnames = [f.replace('.jpg','_words.json') for f in tile_fnames]
        
        images = [keras_ocr.tools.read(f) for f in tile_fnames]
        
        pred_groups = pipeline.recognize(images)
        write_predictions(json_fnames,pred_groups)
        print()

Current: TOP50raster-31O-2018
Recognition preprocess: 1.733 s
Recognition predict: 20.214 s

Current: TOP50raster-32O-2018
Recognition preprocess: 1.697 s
Recognition predict: 12.424 s

Current: TOP50raster-32W-2018
Recognition preprocess: 1.594 s
Recognition predict: 11.085 s

Current: TOP50raster-38O-2018
Recognition preprocess: 2.036 s
Recognition predict: 12.618 s

Current: TOP50raster-39O-2018
Recognition preprocess: 2.131 s
Recognition predict: 13.054 s

Current: TOP50raster-39W-2018
Recognition preprocess: 2.487 s
Recognition predict: 2.809 s

Current: TOP50raster-44O-2018
Recognition preprocess: 2.028 s
Recognition predict: 13.049 s

Current: TOP50raster-45O-2018
Recognition preprocess: 1.91 s
Recognition predict: 2.13 s

Current: TOP50raster-45W-2018
Recognition preprocess: 2.267 s
Recognition predict: 14.396 s



### Visualize predictions on first tile

In [8]:
fname = 'TOP50raster-31O-2018_tile_0_0.jpg'
img = cv2.imread(os.path.join(tile_dir,fname))
words = json.load(open(os.path.join(tile_dir,fname.replace('.jpg','_words.json')), 'r'))['words']

for w in words:
    text = w['text']
    bbox = np.array(w['boundingBox'],dtype=np.int32)
    bbox = bbox.reshape((4,2))
    
    x1, y1 = np.min(bbox,axis=0)-2
    cv2.polylines(img, np.int32([bbox]), True, (0, 0, 255), 3)
    cv2.putText(img, text, (x1, y1), cv2.FONT_HERSHEY_PLAIN, 2, (225, 0, 0), 2, cv2.LINE_AA)
cv2.imwrite('TOP50raster-31O-2018_tile_0_0_detections.jpg',img)

True

### Merge predictions from the tiles

In [10]:
for fname in os.listdir(img_dir):
    if fname.endswith('.jpg'):
        map_name = fname.replace('.jpg','')
        print('Current:',map_name)
        lines_fname = os.path.join(json_dir,fname.replace('.jpg','_lines.json'))
        
        # Check if the map has been fully processed before
        if os.path.exists(lines_fname):
            print('Map already processed')
            continue
        tile_fnames = [os.path.join(tile_dir,f) for f in os.listdir(tile_dir) if map_name in f]
        
        max_r = 0
        max_c = 0

        for tile_name in tile_fnames:
            r = tile_name.split('_tile_')[1][0]
            c = tile_name.split('_tile_' + str(r) + '_')[1][0]

            if int(r) > max_r:
                max_r = int(r)
            if int(c) > max_c:
                max_c = int(c)
        
        # Make 2D matrix to compare adjacent tiles
        words_matrix = [[None] * (max_c + 1) for i in range(max_r + 1)]

        for r in range(max_r + 1):
            for c in range(max_c + 1):
                delta_y = tile_size * r
                delta_x = tile_size * c
                
                pred_fname = os.path.join(tile_dir,map_name + '_tile_' + str(r) + '_' + str(c) + '_words.json')
                words = json.load(open(pred_fname,'r'))['words']
                tile_words = []
                for w in words:
                    bbox = w['boundingBox']
                    # Filter on words inside region without overlap
                    if bbox[0] <= tile_size and bbox[1] <= tile_size:
                        w = map_coords_word(w, delta_x, delta_y)
                        tile_words.append(w)

                words_matrix[r][c] = tile_words

        # Remove symbols to merge text
        pat = re.compile(r'[-.,;:?!_()\[\]{\}&°+%€/]+')
        max_y = 0
        total_words = 0
        for r in range(max_r + 1):
            max_y += tile_size
            max_x = 0
            for c in range(max_c + 1):
                max_x += tile_size
                current_words = words_matrix[r][c]
                total_words += len(current_words)
                current_candidates_right = []
                current_candidates_below = []

                # Filter candidates first to avoid unneccessary loops
                for i, word in enumerate(current_words):
                    bbox = word['boundingBox']

                    if c < max_c and (bbox[2] > max_x or bbox[4] > max_x):
                        current_candidates_right.append((i, word))
                    if r + 1 < max_r and (bbox[5] > max_y or bbox[7] > max_y):
                        current_candidates_below.append((i, word))

                # Check the right
                if c < max_c:
                    next_words = words_matrix[r][c + 1]
                    words_to_remove = []
                    candidates = []
                    # Filter candidates first to avoid unneccessary loops
                    for i, word in enumerate(next_words):
                        bbox = word['boundingBox']

                        if bbox[2] < max_x + overlap or bbox[4] < max_x + overlap:
                            candidates.append((i, word))
                    # TODO: fix code duplication
                    # For each line in current candidates, see if it intersects with a candidate from next tile
                    for current_candidate in current_candidates_right:
                        current_index = current_candidate[0]
                        current_bbox = current_candidate[1]['boundingBox']

                        for candidate in candidates:
                            index = candidate[0]

                            bbox = candidate[1]['boundingBox']

                            if iou(current_bbox, bbox) > 0:
                                current_text = re.sub(pat, '', current_candidate[1]['text'].lower())
                                text = re.sub(pat, '', candidate[1]['text'].lower())
                                partial_ratio = fuzz.partial_ratio(current_text, text)

                                # match, replace text in current by longest of the two,delete line in other
                                if partial_ratio > 60:
                                    # print('match', text, '+', current_text)
                                    words_to_remove.append(index)
                                    # replace current by other
                                    if len(current_text) < len(text):
                                        current_words[current_index] = next_words[index]

                                    break

                    words_matrix[r][c] = current_words
                    for i in sorted(words_to_remove, reverse=True):
                        # print('Deleting', lines_matrix[r][c + 1][i]['text'])
                        del words_matrix[r][c + 1][i]

                # Check the bottom
                if r < max_r:
                    next_words = words_matrix[r + 1][c]
                    words_to_remove = []
                    candidates = []
                    # Filter candidates first to avoid unneccessary loops
                    for i, word in enumerate(next_words):
                        bbox = word['boundingBox']

                        if bbox[5] < max_y + overlap or bbox[7] < max_y + overlap:
                            candidates.append((i, word))

                    # For each line in current candidates, see if it intersects with a candidate from next tile
                    for current_candidate in current_candidates_below:
                        current_index = current_candidate[0]
                        current_bbox = current_candidate[1]['boundingBox']

                        for candidate in candidates:
                            index = candidate[0]
                            bbox = candidate[1]['boundingBox']

                            if iou(current_bbox, bbox) > 0:
                                current_text = re.sub(pat, '', current_candidate[1]['text'].lower())
                                text = re.sub(pat, '', candidate[1]['text'].lower())
                                partial_ratio = fuzz.partial_ratio(current_text, text)

                                # match, replace text in current by longest of the two,delete line in other
                                if partial_ratio > 60:
                                    # print('match', text, '+', current_text)
                                    words_to_remove.append(index)
                                    # replace current by other
                                    if len(current_text) < len(text):
                                        current_words[current_index] = next_words[index]

                                    break

                    words_matrix[r][c] = current_words

                    for i in sorted(words_to_remove, reverse=True):
                        # print('Deleting', lines_matrix[r + 1][c][i]['text'])
                        del words_matrix[r + 1][c][i]

        print('\nAverage words in each tile:', total_words / ((max_c + 1) * (max_r + 1)))
        words = []
        
        for r in range(max_r + 1):
            for c in range(max_c + 1):
                words.extend(words_matrix[r][c])
               
                tile_path = os.path.join(tile_dir,map_name+'_tile_' + str(r) + '_' + str(c) + '.jpg')
                tile_w_path = os.path.join(tile_dir,map_name+'_tile_' + str(r) + '_' + str(c) + '_words.json')
                try:
                    os.remove(tile_path)
                    os.remove(tile_w_path)
                except FileNotFoundError:
                    pass
                
                
        words_path = os.path.join(json_dir,map_name+'_words.json')
        json.dump({'words': words}, open(words_path, 'w'))

Current: TOP50raster-31O-2018

Average words in each tile: 61.05
Current: TOP50raster-32O-2018

Average words in each tile: 59.75
Current: TOP50raster-32W-2018

Average words in each tile: 54.25
Current: TOP50raster-38O-2018

Average words in each tile: 70.45
Current: TOP50raster-39O-2018

Average words in each tile: 73.55
Current: TOP50raster-39W-2018

Average words in each tile: 86.6
Current: TOP50raster-44O-2018

Average words in each tile: 72.1
Current: TOP50raster-45O-2018

Average words in each tile: 66.3
Current: TOP50raster-45W-2018

Average words in each tile: 79.85


In [11]:
def show(img):
    return plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))

def get_rotated_width_height(box):
    """
    Returns the width and height of a rotated bounding box

    Args:
        box: A list of four points starting in the top left
        corner and moving clockwise.
    """
    w = (spatial.distance.cdist(box[0][np.newaxis], box[1][np.newaxis], "euclidean") + spatial.distance.cdist(
        box[2][np.newaxis], box[3][np.newaxis], "euclidean")) / 2
    h = (spatial.distance.cdist(box[0][np.newaxis], box[3][np.newaxis], "euclidean") + spatial.distance.cdist(
        box[1][np.newaxis], box[2][np.newaxis], "euclidean")) / 2

    return w, h


def warpBox(image,
            box,
            target_height=None,
            target_width=None,
            margin=0,
            cval=None,
            return_transform=False):
    """Warp a boxed region in an image given by a set of four points into
    a rectangle with a specified width and height. Useful for taking crops
    of distorted or rotated text.

    Args:
        image: The image from which to take the box
        box: A list of four points starting in the top left
            corner and moving clockwise.
        target_height: The height of the output rectangle
        target_width: The width of the output rectangle
        return_transform: Whether to return the transformation
            matrix with the image.
    """

    if cval is None:
        cval = (0, 0, 0) if len(image.shape) == 3 else 0
    box, _ = get_rotated_box(box)
    w, h = get_rotated_width_height(box)
    if h > 2*w:
        min_x_idx = np.argmin(box[:,0])
        min_x_p = box[min_x_idx]
        max_x_idx = np.argmax(box[:,0])
        max_x_p = box[max_x_idx]
        if min_x_p[1] > max_x_p[1]:
            n_roll = 1
        else:
            n_roll = 3
        temp = h
        h = w
        w = temp
        box = np.roll(box,n_roll,axis=0)
        
       
    assert (
            (target_width is None and target_height is None)
            or (target_width is not None and target_height is not None)), \
        'Either both or neither of target width and height must be provided.'
    if target_width is None and target_height is None:
        target_width = w
        target_height = h
    scale = min(target_width / w, target_height / h)
    M = cv2.getPerspectiveTransform(src=box,
                                    dst=np.array([[margin, margin], [scale * w - margin, margin],
                                                  [scale * w - margin, scale * h - margin],
                                                  [margin, scale * h - margin]]).astype('float32'))
    crop = cv2.warpPerspective(image, M, dsize=(int(scale * w), int(scale * h)))
    target_shape = (target_height, target_width, 3) if len(image.shape) == 3 else (target_height,
                                                                                   target_width)
    full = (np.zeros(target_shape) + cval).astype('uint8')
    full[:crop.shape[0], :crop.shape[1]] = crop
    if return_transform:
        return full, M
    return full


def get_rotated_box(points):
    """Obtain the parameters of a rotated box.

    Returns:
        The vertices of the rotated box in top-left,
        top-right, bottom-right, bottom-left order along
        with the angle of rotation about the bottom left corner.
    """
    try:
        mp = geometry.MultiPoint(points=points)
        pts = np.array(list(zip(*mp.minimum_rotated_rectangle.exterior.xy)))[:-1]  # noqa: E501
    except AttributeError:
        # There weren't enough points for the minimum rotated rectangle function
        pts = points
    # The code below is taken from
    # https://github.com/jrosebr1/imutils/blob/master/imutils/perspective.py

    # sort the points based on their x-coordinates
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (tl, bl) = leftMost

    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    D = spatial.distance.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
    (br, tr) = rightMost[np.argsort(D)[::-1], :]

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    pts = np.array([tl, tr, br, bl], dtype="float32")

    rotation = np.arctan((tl[0] - bl[0]) / (tl[1] - bl[1]))
    return pts, rotation


def get_rect_rotation(polygon):
    bbox = np.array(polygon.minimum_rotated_rectangle.exterior.coords).round().astype(np.int32)
    pts = bbox[:4]
     # sort the points based on their x-coordinates
    xSorted = pts[np.argsort(pts[:, 0]), :]

    # grab the left-most and right-most points from the sorted
    # x-roodinate points
    leftMost = xSorted[:2, :]
    rightMost = xSorted[2:, :]

    # now, sort the left-most coordinates according to their
    # y-coordinates so we can grab the top-left and bottom-left
    # points, respectively
    leftMost = leftMost[np.argsort(leftMost[:, 1]), :]
    (tl, bl) = leftMost

    # now that we have the top-left coordinate, use it as an
    # anchor to calculate the Euclidean distance between the
    # top-left and right-most points; by the Pythagorean
    # theorem, the point with the largest distance will be
    # our bottom-right point
    D = spatial.distance.cdist(tl[np.newaxis], rightMost, "euclidean")[0]
    (br, tr) = rightMost[np.argsort(D)[::-1], :]

    # return the coordinates in top-left, top-right,
    # bottom-right, and bottom-left order
    pts = np.array([tl, tr, br, bl], dtype="float32")

    rotation = np.arctan((tl[0] - bl[0]) / (tl[1] - bl[1]))
    return rotation

def get_nearby_boxes(box,ids_done,polygons,dist=0):
    delta_degrees = 15
    if dist > 0:
        box = box.buffer(dist)
    matches = set()
    box_rotation = np.rad2deg(get_rect_rotation(box))
    for i,p in enumerate(polygons):
        if i in ids_done:
            continue
        # Search for intersecting boxes
        if box.intersects(p):
            # Check if their angles don't differ too much
            rot = np.rad2deg(get_rect_rotation(p))
            if abs(box_rotation - rot) <= delta_degrees:
                matches.add(i)
                
    return matches


def get_nearby_boxes_rec(box,skip_indexes,polygons,dist=0):
    
    total_matches = set()
    # look for matches
    matches = get_nearby_boxes(box,skip_indexes,polygons,dist)
    total_matches |= matches
    skip_indexes |= matches
    while len(matches) > 0:
        next_id = matches.pop()
        next_box = polygons[next_id]
        new_matches = get_nearby_boxes(next_box,skip_indexes,polygons,dist)
        matches |= new_matches
        total_matches |= matches
        skip_indexes |= matches
        
    return total_matches

# Sort left-to-right, top-to-bottom
def sort_key(polygon):
    c = list(polygon.centroid.coords)
    
    return (c[0][0]+c[0][1])/2


def sort_polygons(polygons):
    return sorted(polygons,key=sort_key)


def sort_polygon_ids(polygons):
    # if 2 or less polygons, we sort them as usual
    if len(polygons) <= 2:
        return np.argsort([sort_key(p) for p in polygons])
    
    # if there are more we sort top to bottom, split into groups
    # then sort each group left to right
    
    # probably need to handle the case if text is oriented from bottom left to top right as this will not be correct!
    centers = [list(p.centroid.coords) for p in polygons]
    y_coords = np.array([c[0][1] for c in centers])
    ids = np.arange(len(polygons))
    sorted_ids = np.argsort(y_coords)
    sorted_y = y_coords[sorted_ids]
    ids = ids[sorted_ids]
    curr_id = 0
    delta_y = 10
    groups = []
   
    curr_group = [sorted_ids[0]]
    
    for i in range(len(sorted_y)-1):
        curr = sorted_y[i]
        next_ = sorted_y[i+1]
        if next_ - curr >= delta_y:
            curr_id += 1
            groups.append(curr_group)
            curr_group = [sorted_ids[i+1]]
        else:
            curr_group.append(sorted_ids[i+1])
        
        if i == len(sorted_y)-2:
            groups.append(curr_group)
        

    # flatten list
    res_ids = [id_ for group in groups for id_ in group]
    
    return ids[res_ids]


### Postprocess predictions

#### Detections that have less than 3 chars detected: merge with overlapping ones and recognize again
#### Detetions that have 3 or more chars: merge with overlapping, add spaces in reading direction

#### Sort polygons, sorted in 2 steps: up/down => split in groups then sort each group left to right
#### Merging overlapping detections: check if they are relatively the same direction, don't merge when angles differ 15 degrees or more
#### Ignore detections with numbers in them when merging

In [13]:
for fname in os.listdir(img_dir):
    if fname.endswith('.jpg'):
        if '-Copy' in fname:
            continue
        map_name = fname.replace('.jpg','')
        print('Current file:',map_name)
        lines_fname = os.path.join(json_dir,fname.replace('.jpg','_lines.json'))
        
        # Check if the map has been fully processed before
        if os.path.exists(lines_fname):
            print('Map already processed')
            continue
        
        words_fname = os.path.join(json_dir,map_name+'_words.json')
        words = json.load(open(words_fname,'r'))['words']
        img = cv2.imread(os.path.join(img_dir,fname))
        
        # merged
        out = img.copy()
        ids_done = set()
        polygons = []
        words_filtered = []
        lines = []
        # create polygons, ignore words with numbers, these should not be merged!
        for w in words:
            if any(char.isdigit() for char in w['text']):
                w['recognize'] = 0
                lines.append(w)
            else:
                words_filtered.append(w)
                bbox = np.array(w['boundingBox'],dtype=np.int32).reshape((4,2))
                polygons.append(geometry.Polygon(bbox))
            
            
        for i,w in enumerate(words_filtered):
            recognize = 0
            text = w['text']
            if i in ids_done:
                continue
            # look for nearby words
            ids_done.add(i)
            box = polygons[i]
            matched_ids = get_nearby_boxes_rec(box,ids_done,polygons,dist=0)
            ids_done |= matched_ids
            matched_polygons = [box]
            matched_text = [text]
            if len(matched_ids) > 0:
                
                # filter out based on height   
                for id_ in matched_ids:
                    matched_polygons.append(polygons[id_])
                    matched_text.append(words_filtered[id_]['text'])
                
                # Check length of words, if each word is >= 3 letters, merge
                # Else take longest
                if min(map(len,matched_text)) >= 3:
                    # Sort polygons in reading direction
                    sorted_ids = sort_polygon_ids(matched_polygons)
                    # replace text
                    matched_text = [matched_text[id_] for id_ in sorted_ids]
                    text = ' '.join(matched_text)
                    #print('Joined:',matched_text)
                else:
                    max_len_id = np.argmax([len(m) for m in matched_text])
                    text = matched_text[max_len_id]
                    recognize = 1
                    #print('Took longest:',text,'from:',matched_text)
        
                multi = geometry.MultiPolygon(matched_polygons)
            else:
                multi = box
            bbox = np.array(multi.minimum_rotated_rectangle.exterior.coords).round().astype(np.int32)
            bbox = bbox[:4]
            obj = {'text':text,'boundingBox':bbox.ravel().tolist(),'recognize':recognize}
            lines.append(obj)
        # uncomment to visualize
        '''
        for l in lines:
            text = l['text']
            bbox = np.array(l['boundingBox']).reshape((4,2))
            x,y = bbox[0]-2
            color = (0,225,0)
            cv2.polylines(out, [bbox], True,color=color, thickness=2)
            cv2.putText(out, text, (x,y), cv2.FONT_HERSHEY_SIMPLEX, 1, (225, 0, 0), 2, cv2.LINE_AA)
        
        cv2.imwrite(map_name+'_boxes.jpg',out)
        '''
        
        json.dump({'lines':lines},open(words_fname.replace('_words.json','_lines.json'),'w'))

Current file: TOP50raster-31O-2018
Current file: TOP50raster-32O-2018
Current file: TOP50raster-32W-2018
Current file: TOP50raster-38O-2018
Current file: TOP50raster-39O-2018
Current file: TOP50raster-39W-2018
Current file: TOP50raster-44O-2018
Current file: TOP50raster-45O-2018
Current file: TOP50raster-45W-2018


In [ ]:
import shutil

# advised to make a copy of the recognition results, in case of later errors
lines_fnames = [os.path.join(json_dir,f) for f in os.listdir(json_dir) if '_lines' in f and '-Copy' not in f]
for f in lines_fnames:
    copy_path = f.replace('.json','-Copy1.json')
    if not os.path.exists(copy_path):
        shutil.copyfile(f,copy_path)

### Words detection results have been postprocessed and merged

#### Now we detect the merged results, adjusting for vertical orientation with warpBox. Words wich are vertical and detected, often have a very short length
#### So for each line that has less than 3 chars detected or is vertical, we detect again, adjusted warpBox for vertical in both directions
#### Because we cannot know which of the two strings is gibberish, we save them both and later filter with geocoder step
#### Also run recognize again on small words that were merged

In [15]:
recognizer = keras_ocr.recognition.Recognizer()
target_height = 31
target_width = 200

for fname in os.listdir(img_dir):
    if '-Copy' in fname or '.ipy' in fname:
        continue
    map_name = fname.replace('.jpg','')
    print('Current file:',map_name)
    lines_fname = os.path.join(json_dir,map_name+'_lines.json')
    lines = json.load(open(lines_fname,'r'))['lines']
    skip = False
    for line in lines:
        if 'geolocation' in line:
            skip = True
            break
    
    if skip:
        print('Map already processed')
        continue
    
    img = keras_ocr.tools.read(os.path.join(img_dir,fname))

    for line in lines:
        text = line['text']
        
        if len(text) < 3 or line['recognize']:
            bbox = np.array(line['boundingBox'],dtype=np.int32).reshape((4,2))
            warped = warpBox(img,bbox,target_height=target_height,target_width=target_width)
            bbox, _ = get_rotated_box(bbox)
            w, h = get_rotated_width_height(bbox)
            # Only run prediction again on vertical boxes or merged boxes from previous step
            if line['recognize'] or h > 2*w:
                # prediction on warpbox
                prediction = recognizer.recognize(warped)

                bbox, _ = get_rotated_box(bbox)
                w, h = get_rotated_width_height(bbox)
                
                # If line is vertical, rotate it, predict again and add that prediction as well
                if h > 2*w:
                    inv = warped[::-1,::-1]
                    inv_prediction = recognizer.recognize(inv)
                    prediction = prediction.strip()+'|'+inv_prediction
                    
                line['text'] = prediction
            
            
        json.dump({'lines':lines},open(lines_fname,'w'))

Looking for pretrained_models/crnn_kurapan.h5
Current file: TOP50raster-31O-2018
Current file: TOP50raster-32O-2018
Current file: TOP50raster-32W-2018
Current file: TOP50raster-38O-2018
Current file: TOP50raster-39O-2018
Current file: TOP50raster-39W-2018
Current file: TOP50raster-44O-2018
Current file: TOP50raster-45O-2018
Current file: TOP50raster-45W-2018


### Now perform geocoder requests

In [25]:
def geocode_request_all(query, lat, lng,geonames_key=None,google_key=None,tomtom_key=None, radius=5000, min_score=50):
    """
    Just returns all found values with all geocoders
    """

    def merge(arr):
        for g in arr:
            name = g['name']
            score = fuzz.partial_ratio(query, name.lower())
            if score > min_score:
                result.append((g, score))

    
    # query each geocoder
    result = []
    if geonames_key is not None:
        geo_result = geonames_request(query,key=geonames_key, lat=lat, lng=lng, radius=radius)
        merge(geo_result)
    if google_key is not None:
        google_result = google_geocode_request(query,key=google_key, lat=lat, lng=lng, radius=radius)
        merge(google_result)
    if tomtom_key is not None:
        tomtom_result = tomtom_request(query,key=tomtom_key, lat=lat, lng=lng, radius=radius)
        merge(tomtom_result)


    # Sort results on score, take shortest length if there is a tie
    result = sorted(result, key=lambda x: (x[1], -len(x[0]['name'])), reverse=True)
    return result


def geonames_request(query,key, lat=None, lng=None, radius=5000, max_rows=40):
    query = urllib.parse.quote(query, safe='')
    url = 'http://api.geonames.org/searchJSON?q=' + query + '&lang=local&country=NL&fuzzy=0.3&maxRows=' + \
    str(max_rows) + '&username=' + key
    
    if radius is not None:
        radius = '&radius=' + str(radius / 1000)
        url += radius
    if lat is not None and lng is not None:
        se, nw = get_bbox_from_radius(lat, lng, radius / 1000)
        s = str(se[0])
        e = str(se[1])
        n = str(nw[0])
        w = str(nw[1])
        bounds = '&south=' + s + '&north=' + n + '&west=' + w + '&east=' + e
        url += bounds

    response = requests.get(url)

    geonames_result = response.json()

    result = []

    duplicate = False
    unique = {}

    for i, g in enumerate(geonames_result['geonames']):

        obj = {'lat': '', 'lng': '', 'name': '', 'type': ''}
        type_ = ''
        name = ''

        if 'lat' in g:
            obj['lat'] = float(g['lat'])
        if 'lng' in g:
            obj['lng'] = float(g['lng'])
        # Prefer to use name for local
        if 'name' in g:
            name = g['name']
            obj['name'] = name
        if 'fcl' in g:
            fcl = g['fcl']
            if fcl == 'P':
                type_ = 'locality'
            elif fcl == 'A':
                type_ = 'administrative_area'
            elif fcl == 'H':
                type_ = 'stream,lake'
            elif fcl == 'R':
                type_ = 'road'
            elif fcl == 'L':
                type_ = 'parks,area'
            elif fcl == 'V':
                type_ = 'forest'
            elif fcl == 'T':
                type_ = 'mountain,hill'
            elif fcl == 'S':
                type_ = 'spot,building,farm'
            else:
                type_ = fcl

            obj['type'] = type_

        if name in unique:
            
            idx = unique[name]
            duplicate = result[idx]

            if duplicate['type'] == 'administrative_area' and type_ == 'locality':
                result[idx] = obj
            '''
            elif duplicate['type'] == 'locality' and type_ == 'administrative_area':
                continue
            '''
        else:
            unique[obj['name']] = i

        result.append(obj)

    return result


def tomtom_request(query, key, lat=None, lng=None, radius=5000, idx_set=None):
    """
    Fuzzy geocode query in a specified area
    :param query:
    :param idx_set:
    :param lat:
    :param lng:
    :param radius:
    :return:
    """
    query = urllib.parse.quote(query, safe='')

    url = 'https://api.tomtom.com/search/2/search/' + query + \
          '.json?typeahead=true&countrySet=NL&language=nl-NL&limit=100&minFuzzyLevel=1&maxFuzzyLevel=4&key' \
          '='+key

    if idx_set is None:
        idx_set = 'Geo,Addr,PAD,Str'

    url += '&idxSet=' + idx_set

    if lat is not None and lng is not None:
        url += '&lat=' + str(lat) + '&lon=' + str(lng)
    if radius is not None:
        url += '&radius=' + str(radius)
    try:
        response = requests.get(url)
        tomtom_result = response.json()
    except BaseException as e:
        print('ERROR TOMTOM REQUEST')
        print('url', url)
        print(e)
        return []

    res = []
    try:
        results = tomtom_result['results']
    except KeyError:
        print('KeyError TomTom')
        return []
    
    def lower_first_letter(s):
        return s[:1].lower() + s[1:] if s else ''

    for r in results:
        obj = {}
        type_ = r['type']
        addr = r['address']
        p = r['position']
        try:
            if type_ == 'Geography':
                entity_type = lower_first_letter(r['entityType'])
                if entity_type in addr:
                    name = addr[entity_type]
                elif 'municipalitySubdivision' in addr:
                    name = addr['municipalitySubdivision']
                else:
                    name = addr['freeformAddress']
                    #print('freeformaddres')

            elif type_ == 'Street' or type_ == 'Point Address' or type_ == 'Address Range':
                name = addr['streetName']

            else:
                print('\nTYPEEE', type_, '\n')
                name = 'Unknown Type - TomTom'
                print(r)
        except KeyError:
            continue

        if 'dist' in r:
            obj['dist'] = r['dist']

        obj['lat'] = float(p['lat'])
        obj['lng'] = float(p['lon'])
        obj['name'] = name
        obj['type'] = type_

        res.append(obj)

    return res



def google_geocode_request(query, key, lat=None, lng=None, radius=5000):
    """
    Fuzzy geocoding query in a specified area
    :param query:
    :param lat:
    :param lng:
    :param radius:
    :return:
    """

    query = urllib.parse.quote(query, safe='')
    
    url = 'https://maps.googleapis.com/maps/api/geocode/json?address=' + query + \
          '&language=nl&region=nl&key='+key


    if lat is not None and lng is not None:
        se, nw = get_bbox_from_radius(lat, lng, radius / 1000)
        bounds = str(se[0]) + ',' + str(se[1]) + '|' + str(nw[0]) + ',' + str(nw[1])
        url += '&bounds=' + bounds

    response = requests.get(url)
    results = []
    google_result = response.json()

    def check_int(n):
        try:
            n = int(n)
            return True
        except BaseException as e:
            return False

    for r in google_result['results']:
        addr = r['address_components']
        name = ''
        type_ = ''
        for a in addr:

            # ignore house numbers as they are listed first
            if a['types'] == ['street_number'] or check_int(a['long_name']):
                continue

            name = a['long_name']
            type_ = ','.join(a['types'])
            break

        loc = r['geometry']['location']

        obj = {'lat': loc['lat'], 'lng': loc['lng'], 'name': name, 'type': type_}
        results.append(obj)

    filtered = []
    if lat is not None and lng is not None:
        for r in results:
            lat = r['lat']
            lng = r['lng']

            if lat < se[0] or lat > nw[0] or lng < nw[1] or lng > se[1]:
                pass
            else:
                filtered.append(r)

        return filtered
    else:
        return results

# Radius in km
def get_bbox_from_radius(lat_, lng_, radius):
    """
    Make a bounding box given a coordinate and radius. The bbox will be the outscribed square
    :param lat_:
    :param lng_:
    :param radius:
    :return:
    """
    r = 6378.1
    pi = math.pi
    # length of diagonal of outscribed square
    radius *= (math.sqrt(2) / 2)
    # Southeast corner
    brng = 135 * pi / 180
    lat = lat_ * pi / 180
    lng = lng_ * pi / 180

    def get_lat_lng(brng, lat, lng):
        # Do the math magic
        lat = math.asin(math.sin(lat) * math.cos(radius / r) + math.cos(lat) * math.sin(radius / r) * math.cos(brng))
        lng += math.atan2(math.sin(brng) * math.sin(radius / r) * math.cos(lat),
                          math.cos(radius / r) - math.sin(lat) * math.sin(lat))

        return lat * 180 / pi, lng * 180 / pi

    lat1, lng1 = get_lat_lng(brng, lat, lng)

    # Northwest corner
    brng = 315 * pi / 180
    lat = lat_ * pi / 180
    lng = lng_ * pi / 180

    lat2, lng2 = get_lat_lng(brng, lat, lng)

    return (lat1, lng1), (lat2, lng2)


def line_inside(bbox,line):
    x_min,y_min,x_max,y_max = bbox
    
    bbox = line['boundingBox']
    x = bbox[::2]
    y = bbox[1::2]
    x1 = min(x)
    x2 = max(x)
    y1 = min(y)
    y2 = max(y)
    
    return x1 > x_min and x2 < x_max and y1 > y_min and y2 < y_max


def haversine(lat1,lng1,lat2,lng2):

    lng1, lat1, lng2, lat2 = map(math.radians, [lng1, lat1, lng2, lat2])
    dlon = lng2 - lng1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    return (2 * 6371 * math.asin(math.sqrt(a)))*1000


def get_bbox_height(bbox):
    x1,y1,x2,y2,x3,y3,x4,y4 = bbox
    p1 = np.array([x1,y1])
    p2 = np.array([x2,y2])
    p3 = np.array([x3,y3])
    p4 = np.array([x4,y4])
    
    d1 = np.sqrt(np.sum((p1-p4)**2))
    d2 = np.sqrt(np.sum((p2-p3)**2))
    
    return (d1+d2)/2    


def remove_duplicate_geolocations(geo):
    placename_types = {'country','political','locality','administrative_area','sublocality','sublocality_level_1','geography'}
    road_types = {'street','road','route'}
    
    for g_ in geo:
        g = g_[0]
        if 'type' in g:
            typ = g['type']
            types = typ.lower().split(',')
            found=False
            for t in types:
                if t in placename_types:
                    g['type'] = 'locality'
                    found = True
                    break
                elif t in road_types:
                    g['type'] = 'street'
                    found = True
                    break
    
    res = []
    found_names = {}
    # lat,lng
    found_coords = np.array([[0,0]],dtype=np.float32)
    delta_street = 0.01
    delta_locality = 0.1
    for g_ in geo:
        g = g_[0]
        if 'type' in g:
            typ = g['type']
            name = g['name'].lower()
            lat = g['lat']
            lng = g['lng']
            c1 = np.array([lat,lng],dtype=np.float32)
            
            if name in found_names:
                found_types = found_names[name]
                # check if it is close
                if typ in found_types:
                   
                    dlat =np.min(np.abs(c1[0]-found_coords[:,0]))
                    dlng =np.min(np.abs(c1[1]-found_coords[:,1]))
                    
                    
                    delta = delta_locality if typ == 'locality' else delta_street
                    
                    if dlat > delta and dlng > delta:
                        res.append(g_)
                        found_coords = np.append(found_coords,[c1],axis=0)
                    
                else:
                    found_types.add(typ)
                    res.append(g_)
                    found_coords = np.append(found_coords,[c1],axis=0)
                    
            else:
                found_names[name] = {typ}
                res.append(g_)
                found_coords = np.append(found_coords,[c1],axis=0)
       
    return res


def filter_geo_inside(geolocations,lat_min,lng_min,lat_max,lng_max):
    """
    Returns the geolocations which lie inside the supplied coordinates
    """
    res =[]
    for g in geolocations:
        lat = g[0]['lat']
        lng = g[0]['lng']
        if lat_min < lat < lat_max and lng_min < lng < lng_max:
            res.append(g)
    return res

def take_longest_match(text, geolocations):
    """
    Filter out duplicate matches
    e.g. if text = zandbergstr and matches are zandberg and zandbergstraat (both have 100 partial string similarity)
    it will take zandbergstraat as that contains the longest substring
    If both contain the same substring, both are returned
    """
    #all have same score in arrary
    if len(geolocations) < 2:
        return geolocations
    else:
        idx_longest = -1
        l_text = len(text)
        # check differences in string length, take least diff if unique
        d_lengths = [len(g[0]['name'])-l_text for g in geolocations]
        abs_lengths = np.abs(d_lengths)
        sorted_lengths = sorted(abs_lengths)
        # if 2 or more best ones have same length diff
        if sorted_lengths[0] == sorted_lengths[1]:
            min_ = np.min([d if d >= 0 else l_text for d in d_lengths ])
            return [geolocations[i] for i in range(len(d_lengths)) if d_lengths[i] == min_]
        else:
            min_idx = np.argmin(abs_lengths)
            return [geolocations[min_idx]]

        
def haversine(lat1,lng1,lat2,lng2):

    lng1, lat1, lng2, lat2 = map(math.radians, [lng1, lat1, lng2, lat2])
    dlon = lng2 - lng1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    return (2 * 6371 * math.asin(math.sqrt(a)))*1000        


def overpass_bbox_query(lat1,lng1,lat2,lng2):
    n = str(lat1)
    w = str(lng1)
    s = str(lat2)
    e = str(lng2)
    
    args =  'data=[out:json];(way(' + n + ',' + w + ',' + s + ',' + e + ')[highway];);out%20geom;'
    url = 'https://lz4.overpass-api.de/api/interpreter?' + args
    
    i = 1
    while i < 4:
        try:
            res = requests.get(url).json()
            return res
        except BaseException as e:
            print('Error with OSM query,retrying for the ' + str(i + 1) + 'th time')
            print(e)
            i += 1
            if i < 4:
                time.sleep(2)

    return None


def overpass_query(query,lat1,lng1,lat2,lng2):
   
    n = str(lat1)
    w = str(lng1)
    s = str(lat2)
    e = str(lng2)
    query = urllib.parse.quote(query, safe='')
    args =  'data=[out:json];(way(' + n + ',' + w + ',' + s + ',' + e + ')[highway][name~"^'+query+'",i];);out%20geom;'
    url = 'https://lz4.overpass-api.de/api/interpreter?' + args
    
    i = 1
    while i < 4:
        try:
            res = requests.get(url).json()
            return res
        except BaseException as e:
            print('Error with OSM query,retrying for the ' + str(i + 1) + 'th time')
            print(e)
            i += 1
            if i < 4:
                time.sleep(2)

    return None

### For vertical text, perform request for both orientations

### Save queries and results in geocode_dict, so no queries are executed twice.

In [ ]:
geocode_fname = os.path.join(data_dir,'geocode_dict.json')
geocode_dict = json.load(open(geocode_fname,'r',encoding='utf-8'))['dict']

# Put your API key(s) here
geonames_key = None
google_key = None
tomtom_key = None

for fname in os.listdir(img_dir):
    if '-Copy' in fname or '.ipy' in fname:
        continue
    map_name = fname.replace('.jpg','')
    lines_fname = os.path.join(json_dir,map_name+'_lines.json')
    lines = json.load(open(lines_fname,'r'))['lines']
    print('Current file:',fname)
    
    for line in lines:
       
        
        text = line['text']
        if 'geolocation' in line:
            continue
        # Filter out numbers    
        try:
            z = int(text)
            continue
        except:
            pass
        
        if len(text) > 3:
            # If text was vertical and recognized twice
            # split and perform 2 geocoder requests
            if '|' in text:
                t1 = text.split('|')[0]
                t2 = text.split('|')[1]
                if t1 not in geocode_dict:
                    g1 = geocode_request_all(t1,geonames_key=geonames_key,google_key=google_key,tomtom_key=tomtom_key,
                                             lat=None,lng=None)
                    geocode_dict[t1] = g1
                    time.sleep(0.2)
                else:
                    g1 = geocode_dict[t1]
                    
                if t2 not in geocode_dict:
                    g2 = geocode_request_all(t2,geonames_key=geonames_key,google_key=google_key,tomtom_key=tomtom_key,
                                             lat=None,lng=None)
                    geocode_dict[t2] = g2
                    time.sleep(0.2)
                else:
                    g2 = geocode_dict[t2]
                    
                line['geolocation'] = g1
                line['geolocation2'] = g2
            else:
                if text not in geocode_dict:
                    geo = geocode_request_all(text,geonames_key=geonames_key,google_key=google_key,tomtom_key=tomtom_key,
                                              lat=None,lng=None)
                    geocode_dict[text] = geo
                    time.sleep(0.2)
                else:
                    geo = geocode_dict[text]
                    
                line['geolocation'] = geo
                

            json.dump({'dict':geocode_dict},open(geocode_fname,'w',encoding='utf-8'))
            json.dump({'lines':lines},open(lines_fname,'w'))